##### **Some introductions**

I've somehow promised myself to do a tree series on machine learning and glad I've made it to the boosted trees part (it took a while...). This is also likely my last post on this topic for now as there are other things I want to explore in the near future. Hopefully this is somewhat useful for anyone who's new to this.

<br>

###### **AdaBoost**

Adaptive Boost or AdaBoost has originated from Robert E. Schapire in 1990 [@schapire1990], [@mlbook2022], and was further introduced in 1996 by Robert Schapire and Yoav Freund at a conference which also led to a publication [@freund1997].

As quoted from [*scikit-learn*](https://scikit-learn.org/stable/modules/ensemble.html#adaboost), an AdaBoost algorithm is doing this:

> ...fit a sequence of weak learners (i.e., models that are only slightly better than random guessing, such as small decision trees) on repeatedly modified versions of the data.

::: {.callout-note}
Weak learner means an ensemble of very simple base classifiers such as decision tree stumps [@mlbook2022]
:::

During the process of running the algorithm, increased weights are given to the incorrectly predicted samples at each iteration, and less weights are given to the correctly predicted ones. This then forces the AdaBoost models to focus more on the less accurately predicted samples with the aim to improve ensemble performance. The predictions from these iterations are combined to produce a final prediction via a weighted majority vote style, which is a well-known signature of tree models. Overall, AdaBoost algorithm can be used for classification or regression problems. The main difference between bagging and boosting is that boosting only uses random subsets of training samples drawn from the training dataset without any replacements [@mlbook2022]. One caveat to note is that AdaBoost tend to overfit training data (high variance).

Parameters to tune:

* *n_estimators* - number of weak learners

* *learning_rate* - contributions of weak learners in the final combination

* *max_depth* - depth of trees

* *min_samples_split* - minimum required number of samples to consider a split

<br>

###### **Gradient boosted trees**

Essentially a similar concept is behind gradient boosted trees where a series of weak learners is trained in order to create a stronger ensemble of models [@mlbook2022]. However, some differences between these two types of boosted trees (e.g. AdaBoost and XGBoost) should be noted, and rather than describing them in a paragraph, I've summarised them in a table below.

In [1]:
#| echo: false
#| tbl-cap-location: margin
#| tbl-cap: Differences between XGBoost and AdaBoost [@mlbook2022]


from IPython.display import Markdown
from tabulate import tabulate

table = [
    ["trains weak learners based on errors from previous decision tree stump", 
    "trains weak learners that are deeper than decision tree stumps with a max depth of 3 to 6 (or max number of leaf nodes from 8 to 64)"], 
    ["uses prediction errors to calculate sample weights and classifier weights", 
    "uses prediction errors directly to produce the target variable to fit the next tree"], 
    ["uses individual weighting terms for each tree", 
    "uses a global learning rate for each tree"]
    ]

Markdown(tabulate(table, headers=["AdaBoost", "XGBoost"]))

AdaBoost                                                                   XGBoost
-------------------------------------------------------------------------  ------------------------------------------------------------------------------------------------------------------------------------
trains weak learners based on errors from previous decision tree stump     trains weak learners that are deeper than decision tree stumps with a max depth of 3 to 6 (or max number of leaf nodes from 8 to 64)
uses prediction errors to calculate sample weights and classifier weights  uses prediction errors directly to produce the target variable to fit the next tree
uses individual weighting terms for each tree                              uses a global learning rate for each tree

XGBoost or extreme gradient boosting [@DBLP:journals/corr/ChenG16] is one of the most commonly used open-source packages, originally developed at the University of Washington by T. Chen and C. Guestrin, that uses stochastic gradient boosting to build an ensemble of predictive models. 

[XGBoost documentation](https://xgboost.readthedocs.io/en/stable/index.html) - https://xgboost.readthedocs.io/en/stable/index.html

Main parameters to tune as suggested by [@bruce2020]: 

- *subsample* - controls fraction of observations that should be sampled at each iteration or a subsample ratio of the training instance (as per [XGBoost's *scikit-learn* API](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)). This is similar to how a random forest operates but without the sample replacement part

- *eta* (in XGBoost) or *learning_rate* (in *scikit-learn* wrapper interface for XGBoost) - a shrinkage factor applied to alpha (a factor derived from weighted errors) in the boosting algorithm or it simply may be more easily understood as the boosting learning rate used to prevent overfitting

There are of course a whole bunch of other [XGBoost parameters](https://xgboost.readthedocs.io/en/latest/parameter.html#xgboost-parameters) that can be tuned, and in order to keep this post at a reasonable reading length, I won't go through every single one of them, but see this link as an example parameter set for [XGBClassifier()](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier).

In *scikit-learn*, there are also two types of gradient boosted tree methods, GradientBoostingClassifer() and HistGradientBoostingClassifier(), in its sklearn.ensemble module (note: equivalent regressor class also available). One way to choose between them is to check sample size first. [GradientBoostingClassifer()](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier) class is likely better when there is only a small sample size (e.g. when number of sample is less than 10,000), while [HistGradientBoostingClassifier()](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html#sklearn.ensemble.HistGradientBoostingClassifier) class is likely better when your sample size is at least 10,000+ or more. The HistGradientBoostingClassifier() is a histogram-based gradient boosting classification tree that is mainly inpired by [LightGBM](https://github.com/Microsoft/LightGBM). 

<br>

##### **A demo**

In the example below, I'm only using AdaBoost classifier and XGBoost classifier for now. Please note that the dataset used here is very small and the example is likely not going to reflect real-life use case completely (use with care). 

<br>

###### **Import libraries**

In [2]:
import sys
import pandas as pd
import numpy as np
import chembl_downloader
import rdkit
from rdkit import Chem
import pickle

# Import Scikit_mol
## Check and clean SMILES
from scikit_mol.utilities import CheckSmilesSanitazion
## Standardise molecules
from scikit_mol.standardizer import Standardizer
## Import fingerprints & descriptors
from scikit_mol.fingerprints import MorganFingerprintTransformer
from scikit_mol.descriptors import MolecularDescriptorTransformer
## Import smi2mol transformer
from scikit_mol.conversions import SmilesToMolTransformer

# Import scikit-learn
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
## Data scaler (variance scaling)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

# Import xgboost classifier
import xgboost
from xgboost import XGBClassifier

print(f"xgboost version used: {xgboost.__version__}")
print(f"scikit-learn version used: {sklearn.__version__}")
print(f"rdkit version used: {rdkit.__version__}")

xgboost version used: 2.0.3
scikit-learn version used: 1.4.2
rdkit version used: 2023.09.5


In [3]:
# Show Python version used 
print(sys.version)

3.11.0 (v3.11.0:deaf509e8f, Oct 24 2022, 14:43:23) [Clang 13.0.0 (clang-1300.0.29.30)]


<br>

###### **Data source**

Data source is based on ChEMBL database version 33 (as shown by the file name below, "chembl_d_ache_33"), which was downloaded previously from last post (on [random forest classifier](https://jhylin.github.io/Data_in_life_blog/posts/17_ML2-2_Random_forest/2_random_forest_classifier.html)) by using [chembl_downloader](https://github.com/cthoyt/chembl-downloader).

In [4]:
from pathlib import Path

# Pick any directory, but make sure it's relative to your home directory
directory = Path.home().joinpath(".data", "blog")
# Create the directory if it doesn't exist
directory.mkdir(exist_ok=True, parents=True)

# Create a file path that corresponds to the previously cached ChEMBL data 
path = directory.joinpath(f"chembl_d_ache_33.tsv")

# alternative way to download latest ChEMBL version
# please see post link - https://jhylin.github.io/Data_in_life_blog/posts/17_ML2-2_Random_forest/2_random_forest_classifier.html#data-retrieval-using-chembl_downloader for details
# note: need to specify late_version = latest() first
# path = directory.joinpath(f"chembl_d_ache_{latest_version}.tsv")

if path.is_file():
    # If the file already exists, load it
    df_ache = pd.read_csv(path, sep=',')
else:
    # If the file doesn't already exist, make the query then cache it
    df_ache = chembl_downloader.query(sql)
    df_ache.to_csv(path, sep=",", index=False)

In [5]:
print(df_ache.shape)
df_ache.head()

(7144, 9)


,assay_chembl_id,target_type,tax_id,chembl_id,canonical_smiles,molecule_chembl_id,max_phase,standard_type,pchembl_value
0,CHEMBL1909212,SINGLE PROTEIN,9606,CHEMBL220,CC/C(=C(/CC)c1ccc(O)cc1)c1ccc(O)cc1,CHEMBL411,4.0,IC50,4.59
1,CHEMBL1003053,SINGLE PROTEIN,9606,CHEMBL220,COc1c2occc2cc2ccc(=O)oc12,CHEMBL416,4.0,IC50,4.27
2,CHEMBL2406149,SINGLE PROTEIN,9606,CHEMBL220,COc1c2occc2cc2ccc(=O)oc12,CHEMBL416,4.0,IC50,6.12
3,CHEMBL1909212,SINGLE PROTEIN,9606,CHEMBL220,CC1(COc2ccc(CC3SC(=O)NC3=O)cc2)CCCCC1,CHEMBL7002,-1.0,IC50,4.82
4,CHEMBL3071788,SINGLE PROTEIN,9606,CHEMBL220,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,CHEMBL28,NaN,IC50,7.92


Exploring max phases of compounds in the dataset.

In [6]:
df_ache.value_counts("max_phase", dropna=False)

max_phase
 NaN    6410
 4.0     618
-1.0      69
 2.0      29
 3.0      10
 1.0       7
 0.5       1
Name: count, dtype: int64

<br>

##### **Datasets**

The [definition of "NaN" assigned to max_phase](https://chembl.gitbook.io/chembl-interface-documentation/frequently-asked-questions/drug-and-compound-questions#what-is-max-phase) indicates that compounds labelled as "NaN" or "null" have no evidence of showing they've reached clinical trials yet, but I'm still keeping them in the dataset (this can also be excluded depending on project goals). 

A max_phase of -1 is assigned to molecules with unknown clinical phase status ([ChEMBL reference](https://chembl.gitbook.io/chembl-interface-documentation/frequently-asked-questions/drug-and-compound-questions#what-is-max-phase)), for which I'll drop for this particular experiment.

In [7]:
# Fill "NaNs" as "0" first
df_ache.fillna({"max_phase": 0}, inplace=True)

In [8]:
# Select only mols with max_phase of 0 and above
df_ache = df_ache[(df_ache["max_phase"] >= 0)]

In [9]:
df_ache["max_phase"].describe()

count    7075.000000
mean        0.362898
std         1.138520
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         4.000000
Name: max_phase, dtype: float64

In [10]:
df_ache.value_counts("max_phase", dropna=False)

max_phase
0.0    6410
4.0     618
2.0      29
3.0      10
1.0       7
0.5       1
Name: count, dtype: int64

In [11]:
# Convert max_phase from float to int for the ease of reading predicted outcomes,
# otherwise it'll look like "4., 2., 4., ..."
df_ache = df_ache.astype({"max_phase": int, "canonical_smiles": "string"})

In [12]:
df_ache.dtypes

assay_chembl_id               object
target_type                   object
tax_id                         int64
chembl_id                     object
canonical_smiles      string[python]
molecule_chembl_id            object
max_phase                      int64
standard_type                 object
pchembl_value                float64
dtype: object

Please note: the only molecule with max_phase of "0.5" was converted into "0" after I've converted the datatype of max_phase from float to integer (I've left it deliberately like this since this is only a demonstration on using scikit_learn's pipeline along with scikit_mol, but in reality this should be handled with care i.e. don't discard it as different max phase values have different meanings!). Therefore the following max_phase value count will have 6411 molecules with max_phase "0", rather than the previous number of 6410.

In [13]:
df_ache.value_counts("max_phase", dropna=False)

max_phase
0    6411
4     618
2      29
3      10
1       7
Name: count, dtype: int64

<br>

##### **Model building using *scikit-learn*'s pipeline**

Binary classification has been used in my previous posts, e.g. target as max_phase 4 (re-labelled as "1") with training set of max_phase "null" re-labelled as "0" with their different RDKit molecular features. This time I'll be using multi-class classification to predict a training set of molecules containing max_phase of 0, 1, 2, 3 and 4.

<br>

###### **Define X and y variables**

In [14]:
# A sanity check and view on the original dataset 
df_ache

,assay_chembl_id,target_type,tax_id,chembl_id,canonical_smiles,molecule_chembl_id,max_phase,standard_type,pchembl_value
0,CHEMBL1909212,SINGLE PROTEIN,9606,CHEMBL220,CC/C(=C(/CC)c1ccc(O)cc1)c1ccc(O)cc1,CHEMBL411,4,IC50,4.59
1,CHEMBL1003053,SINGLE PROTEIN,9606,CHEMBL220,COc1c2occc2cc2ccc(=O)oc12,CHEMBL416,4,IC50,4.27
2,CHEMBL2406149,SINGLE PROTEIN,9606,CHEMBL220,COc1c2occc2cc2ccc(=O)oc12,CHEMBL416,4,IC50,6.12
4,CHEMBL3071788,SINGLE PROTEIN,9606,CHEMBL220,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,CHEMBL28,0,IC50,7.92
5,CHEMBL1119333,SINGLE PROTEIN,9606,CHEMBL220,CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1,CHEMBL41,4,IC50,6.89
...,...,...,...,...,...,...,...,...,...
7139,CHEMBL5216374,SINGLE PROTEIN,9606,CHEMBL220,CCC(C)(C)NCC(O)c1cc(O)cc(OC(=O)N(C)C)c1,CHEMBL5220560,0,IC50,4.70
7140,CHEMBL5216425,SINGLE PROTEIN,9606,CHEMBL220,CC(C)(C)OC(=O)Nc1ccc(O)c(C(=O)NCCCN2CCCCC2)c1,CHEMBL5220695,0,Ki,6.92
7141,CHEMBL5216408,SINGLE PROTEIN,9606,CHEMBL220,O=C1CCOc2cc(OCCCCCSC(=S)N3CCCCC3)ccc21,CHEMBL5220742,0,IC50,7.00
7142,CHEMBL5218078,SINGLE PROTEIN,9606,CHEMBL220,Cn1ccc2cc(-c3cnc4ccc(C(=O)N5CCCCC5)cc4n3)ccc2c1=O,CHEMBL5220884,0,IC50,5.27


In [15]:
X = df_ache.canonical_smiles
y = df_ache.max_phase

In [16]:
X.shape

(7075,)

In [17]:
y.shape

(7075,)

In [18]:
print(X)

0                     CC/C(=C(/CC)c1ccc(O)cc1)c1ccc(O)cc1
1                               COc1c2occc2cc2ccc(=O)oc12
2                               COc1c2occc2cc2ccc(=O)oc12
4                    O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12
5                      CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1
                              ...                        
7139              CCC(C)(C)NCC(O)c1cc(O)cc(OC(=O)N(C)C)c1
7140        CC(C)(C)OC(=O)Nc1ccc(O)c(C(=O)NCCCN2CCCCC2)c1
7141               O=C1CCOc2cc(OCCCCCSC(=S)N3CCCCC3)ccc21
7142    Cn1ccc2cc(-c3cnc4ccc(C(=O)N5CCCCC5)cc4n3)ccc2c1=O
7143       COC(=O)c1cc2c(OC(=O)N(C)C)cccc2[n+]([11CH3])c1
Name: canonical_smiles, Length: 7075, dtype: string


In [19]:
print(y)

0       4
1       4
2       4
4       0
5       4
       ..
7139    0
7140    0
7141    0
7142    0
7143    0
Name: max_phase, Length: 7075, dtype: int64


<br>

###### **Sanitise SMILES**

This post is going to focus on Scikit_mol which has a manual way to handle SMILES errors, as shown in the code below. Another useful way to deal with SMILES errors is [Molpipeline](https://github.com/basf/MolPipeline)'s SMILES error handling, with an example shown in one of [its notebooks](https://github.com/basf/MolPipeline/blob/main/notebooks/03_error_handling.ipynb). The main difference from what I can see, even though I haven't got to use it yet, is that Molpipeline takes into account of all the invalid SMILES by giving each invalid SMILES a "NaN" label in the pipeline process - this maintains the matrix shape and good for tracking down the problematic SMILES (molecules).

In [20]:
checksmi = CheckSmilesSanitazion()
# Checking on SMILES (X) 
X_valid, X_errors = checksmi.sanitize(X)

Ideally, X_valid & y_valid will be used for further work, meaning all the invalids or errors will be removed from the training set.

In [21]:
checksmi.errors

,SMILES


No SMILES errors are shown.

In [22]:
X_errors

[]

No outputs are there (no errors detected). 

In [23]:
print(X_valid)

['CC/C(=C(/CC)c1ccc(O)cc1)c1ccc(O)cc1', 'COc1c2occc2cc2ccc(=O)oc12', 'COc1c2occc2cc2ccc(=O)oc12', 'O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12', 'CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1', 'CCN(CC)CCCC(C)Nc1c2ccc(Cl)cc2nc2ccc(OC)cc12', 'O=c1c(O)c(-c2ccc(O)c(O)c2)oc2cc(O)cc(O)c12', 'O=C1C=CC(=O)C=C1', 'CCc1c2c(nc3ccc(OC(=O)N4CCC(N5CCCCC5)CC4)cc13)-c1cc3c(c(=O)n1C2)COC(=O)[C@]3(O)CC', 'CCc1c2c(nc3ccc(OC(=O)N4CCC(N5CCCCC5)CC4)cc13)-c1cc3c(c(=O)n1C2)COC(=O)[C@]3(O)CC', 'O=C(c1ccc(OCCN2CCCCC2)cc1)c1c(-c2ccc(O)cc2)sc2cc(O)ccc12', 'Oc1c(Cl)cc(Cl)c(Cl)c1Cc1c(O)c(Cl)cc(Cl)c1Cl', 'Clc1ccc(COC(Cn2ccnc2)c2ccc(Cl)cc2Cl)c(Cl)c1', 'CNC(=O)Oc1ccc2c(c1)OC[C@@H]1CCN(C)[C@H]21', 'CNC(=O)Oc1ccc2c(c1)OC[C@@H]1CCN(C)[C@H]21', 'CNC(=O)Oc1ccc2c(c1)[C@]1(C)CCN(C)[C@@H]1N2C', 'CNC(=O)Oc1ccc2c(c1)[C@]1(C)CCN(C)[C@@H]1N2C', 'CNC(=O)Oc1ccc2c(c1)[C@]1(C)CCN(C)[C@@H]1N2C', 'CNC(=O)Oc1ccc2c(c1)[C@]1(C)CCN(C)[C@@H]1N2C', 'CNC(=O)Oc1ccc2c(c1)[C@]1(C)CCN(C)[C@@H]1N2C', 'CNC(=O)Oc1ccc2c(c1)[C@]1(C)CCN(C)[C@@H]1N2C', 'CNC(=O)Oc1ccc2c(

Also, manually checking for any "NaNs" in the canonical SMILES column (X variable), since AdaBoost classifier won't accept missing values in the dataset, but if using HistGradientBoostingClassifier() instead, it should take care of the native NaNs.

In [24]:
print(f"{df_ache.canonical_smiles.isna().sum()} out of {len(df_ache)} SMILES failed in conversion")

0 out of 7075 SMILES failed in conversion


There are other ways to deal with NaNs with a few examples provided by [*scikit-learn*](https://scikit-learn.org/stable/modules/impute.html). However, with regards to drug discovery data, there are probably more caveats that need to be taken during data preprocessing (I'm also still exploring this too). 

<br>

###### **Split data**

Randomly splitting data this time.

In [25]:
# Found a silly error when naming X, y train/test sets!
# Remember to name them in this order: X_train, X_test, y_train, y_test
# otherwise model fitting won't work...
X_train, X_test, y_train, y_test = train_test_split(X_valid, y, test_size=0.2, random_state=3)

<br>

##### **Create pipelines**

The aim is to create pipeline(s) using *scikit-learn*.

<br>

###### **AdaBoost classifier**

The original plan is to chain an AdaBoost classifier, XGBoost classifier, along with Scikit-mol transformers all at once. However, it turns out that I'm building two separate pipelines of AdaBoost classifier and XGBoost classifier so that I can compare the difference(s) between them, and this also serves better for the purpose of this post really.

This is also the time to think about generating molecular features for model training. Choosing data features such as fingerprints (e.g. Morgan fingerprints which is usually best for larger dataset) or RDKit 2D descriptors (which is useful for smaller datasets) or others. For RDKit 2D descriptors, Scikit_mol has integrated RDKit's [rdkit.Chem.Descriptors module](https://www.rdkit.org/docs/source/rdkit.Chem.Descriptors.html#module-rdkit.Chem.Descriptors) and [rdkit.ML.Descriptors.MoleculeDescriptors module](https://www.rdkit.org/docs/source/rdkit.ML.Descriptors.MoleculeDescriptors.html#rdkit.ML.Descriptors.MoleculeDescriptors.MolecularDescriptorCalculator) within its MolecularDescriptorTransformer().

Some useful links regarding building pipelines in *scikit-learn* and also another reference notebook on when to use parallel calculations for different molecular features:

- [Pipeline module](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn-pipeline-pipeline)

- [make_pipeline module](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html#sklearn-pipeline-make-pipeline) (simpler pipeline construction, without naming estimators ourselves but rather naming them automatically)

- [Useful notebook](https://github.com/EBjerrum/scikit-mol/blob/main/notebooks/07_parallel_transforms.ipynb) explaining when is the best to run parallel calculations to calculate molecular fingerprints and descriptors

Initally, RDKit 2D descriptors are used and to see all the available descriptors, run the following code.

In [26]:
rdkit2d = MolecularDescriptorTransformer()
available_descriptors = rdkit2d.available_descriptors
print(f"First 10 descriptor names: {available_descriptors[:10]}")

First 10 descriptor names: ['MaxAbsEStateIndex', 'MaxEStateIndex', 'MinAbsEStateIndex', 'MinEStateIndex', 'qed', 'SPS', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons']


For the first sample pipeline I'm building below, I've noticed that not all of the 209 RDKit 2D descriptors can be used for AdaBoost classifier, as some of the descriptors will have values of "0" and AdaBoost classifier will not be able to take care of them. Therefore, I'm only using a small selection of descriptors only, but HistGradientBoostingClassifier() should be able to take into account of NaNs and can be chained to include all descriptors in a pipeline.

The following is an example of building a *scikit_learn* pipeline by using AdaBoost classifier model, along with Scikit_mol's transformers for multi-class max_phase predictions with training set consisting of molecules with max_phase 0, 1, 2, 3 and 4. I've used Morgan fingerprints instead eventually so that'll be shown in the following pipeline code, but I've also kept the RDKit 2D descriptor option on (just need to uncomment to run).

In [27]:
# Set parameters for RDKit 2D descriptors
# params_rdkit2d = {
#     "desc_list": ['HeavyAtomCount', 'FractionCSP3', 'RingCount', 'MolLogP', 'MolWt']
# }

# Set parameters for adaboost model
params_adaboost = {
    "estimator": DecisionTreeClassifier(max_depth = 3), 
    # default: n_estimators = 50, learning_rate = 1.0 (trade-off between them)
    "n_estimators": 80, 
    "learning_rate": 0.2, 
    # SAMME (Stagewise Additive Modeling using a Multi-class Exponential loss function) algorithm 
    # for multi-class classification
    "algorithm": "SAMME", 
    "random_state": 2,
    }

# Building AdaBoostClassifier pipeline
mlpipe_adaboost = make_pipeline(
    # Convert SMILES to RDKit molecules
    SmilesToMolTransformer(), 
    # Molecule standardisations
    Standardizer(),
    ## A choice of using either Morgan fingerprints or RDKit 2D descriptors:
    # Generate MorganFingerprintTransformer()
    MorganFingerprintTransformer(useFeatures=True),
    # Generate RDKit2D descriptors
    #MolecularDescriptorTransformer(**params_rdkit2d),
    # Scale variances in descriptor data
    StandardScaler(),
    # Apply adaptive boost classifier
    AdaBoostClassifier(**params_adaboost)
)

In [28]:
# Check on pipeline
mlpipe_adaboost

Pipeline(steps=[('smilestomoltransformer', SmilesToMolTransformer()),
                ('standardizer', Standardizer()),
                ('morganfingerprinttransformer',
                 MorganFingerprintTransformer(useFeatures=True)),
                ('standardscaler', StandardScaler()),
                ('adaboostclassifier',
                 AdaBoostClassifier(algorithm='SAMME',
                                    estimator=DecisionTreeClassifier(max_depth=3),
                                    learning_rate=0.2, n_estimators=80,
                                    random_state=2))])

An interactive pipeline diagram should show with a status of "not fitted" if you hover the mouse over the "i" logo on top right. The pipeline is then fitted onto the training sets.

In [29]:
mlpipe_adaboost.fit(X_train, y_train)

Pipeline(steps=[('smilestomoltransformer', SmilesToMolTransformer()),
                ('standardizer', Standardizer()),
                ('morganfingerprinttransformer',
                 MorganFingerprintTransformer(useFeatures=True)),
                ('standardscaler', StandardScaler()),
                ('adaboostclassifier',
                 AdaBoostClassifier(algorithm='SAMME',
                                    estimator=DecisionTreeClassifier(max_depth=3),
                                    learning_rate=0.2, n_estimators=80,
                                    random_state=2))])

The pipeline status should now show a "fitted" message if hovering over the same "i" logo. Then the pipeline is used on the X_test (testing X set) to predict the target (max_phase) variable.

In [30]:
mlpipe_adaboost.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

<br>

###### **XGBoost classfier**

The following code snippet is an example of a scikit_learn pipeline using Scikit_mol's transformers and XGBoost classifier. One nice thing about XGBoost is that is has [*scikit_learn* interface](https://xgboost.readthedocs.io/en/stable/python/sklearn_estimator.html) so that it can be integrated into the *scikit-learn* pipeline and Scikit_mol's transformers, which is what I've tried below.

In [31]:
# Set parameters for xgboost model
params_xgboost = {
    "n_estimators": 100,
    "max_depth": 3,
    # For multi-class classification, use softprob for loss function (learning task parameters)
    # source: https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters
    "objective": 'multi:softprob', 
    "learning_rate": 0.1, 
    "subsample": 0.5, 
    "random_state": 2
    }

# Building XGBoostClassifier pipeline
mlpipe_xgb = make_pipeline(
    # Convert SMILES to RDKit molecules
    SmilesToMolTransformer(), 
    # Molecule standardisations
    Standardizer(),
    ## A choice of using either Morgan fingerprints  or RDKit 2D descriptors:
    # Generate MorganFingerprintTransformer()
    MorganFingerprintTransformer(useFeatures=True),
    # Generate RDKit2D descriptors
    #MolecularDescriptorTransformer(**params_rdkit2d),
    # Scale variances in descriptor data
    StandardScaler(),
    # XGBoost classifier
    XGBClassifier(**params_xgboost)
)

In [32]:
mlpipe_xgb

Pipeline(steps=[('smilestomoltransformer', SmilesToMolTransformer()),
                ('standardizer', Standardizer()),
                ('morganfingerprinttransformer',
                 MorganFingerprintTransformer(useFeatures=True)),
                ('standardscaler', StandardScaler()),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytr...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=3, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=100, n_jobs=None,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

In [33]:
mlpipe_xgb.fit(X_train, y_train)

Pipeline(steps=[('smilestomoltransformer', SmilesToMolTransformer()),
                ('standardizer', Standardizer()),
                ('morganfingerprinttransformer',
                 MorganFingerprintTransformer(useFeatures=True)),
                ('standardscaler', StandardScaler()),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytr...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=3, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=100, n_jobs=None,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

In [34]:
pred = mlpipe_xgb.predict(X_test)
pred

array([0, 0, 0, ..., 0, 0, 0])

<br>

###### **Model metrics**

One can never just leave the process of building a machine learning model without evaluating it. Although what I have done below is probably very minimal but it's somewhat a starting point to think about how good the model is.

In [35]:
from sklearn.metrics import accuracy_score

# Following misclassification score function code borrowed and adapted from:
# https://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_multiclass.html#sphx-glr-auto-examples-ensemble-plot-adaboost-multiclass-py

def misclassification_error(y_true, y_pred):
    return 1 - accuracy_score(y_true, y_pred)

mlpipe_adaboost_misclassification_error = misclassification_error(
    y_test, mlpipe_adaboost.fit(X_train, y_train).predict(X_test)
)

mlpipe_xgb_misclassifiaction_error = misclassification_error(
    y_test, mlpipe_xgb.fit(X_train, y_train).predict(X_test)
)

print(f"Training score for mlpipe_adaboost: {mlpipe_adaboost.score(X_train, y_train):0.2f}")
print("Testing score for mlpipe_adaboost: "f" {mlpipe_adaboost.score(X_test, y_test):0.2f}")
print("AdaBoostClassifier's misclassification_error: "f"{mlpipe_adaboost_misclassification_error:0.3f}")

print(f"Training score for mlpipe_xgb: {mlpipe_xgb.score(X_train, y_train):0.2f}")
print("Testing score for mlpipe_xgb: "f"{mlpipe_xgb.score(X_test, y_test):0.2f}")
print("XGBClassifier's missclassification_error: "f"{mlpipe_xgb_misclassifiaction_error:0.3f}")

Training score for mlpipe_adaboost: 0.97
Testing score for mlpipe_adaboost:  0.97
AdaBoostClassifier's misclassification_error: 0.028
Training score for mlpipe_xgb: 0.99
Testing score for mlpipe_xgb: 0.99
XGBClassifier's missclassification_error: 0.014


It appears that XGBoost model obtained a better prediction accuracy than the AdaBoost one (although the models are built in a very simple way, but it still shows the slight difference in performance). The training data being used here is also very imbalanced with a lot of them being max_phase of "0" than "4", and with max_phase "4" being our ultimate aim, the dataset used above is really for demonstration only. Also, since this post is already quite long, I'd rather not make this post into a gigantic tl;dr, so for the imbalanced data discussion and exploration, my previous posts have tried to touch on this topic - ["Random forest"](https://jhylin.github.io/Data_in_life_blog/posts/17_ML2-2_Random_forest/1_random_forest.html) and ["Random forest classifier"](https://jhylin.github.io/Data_in_life_blog/posts/17_ML2-2_Random_forest/2_random_forest_classifier.html).

<br>

###### **Hyperparameter tuning for XGBoost classifier**

For XGBoost, one of the main things is to minimise model overfitting where several parameters will play important roles to achieve this. For example, *learning_rate* and *subsample* are the first two mentioned previously, and another technique is based on regularisation which includes two other parameters, *reg_alpha* (L1 regularisation based on Manhattan distance) and *reg_lamda* (L2 regularisation based on Euclidean distance). Both of these regularisation parameters aim to penalise XGBoost's model complexity to make it a bit more conservative in order to reduce overfitting [@bruce2020].

A full list of XGBoost classifier pipeline parameters and settings used can be retrieved as shown below. It contains a **long** list of parameters and one of the ways to find the optimal set of parameters is by using cross-validation (CV).

In [36]:
mlpipe_xgb.get_params()

{'memory': None,
 'steps': [('smilestomoltransformer', SmilesToMolTransformer()),
  ('standardizer', Standardizer()),
  ('morganfingerprinttransformer',
   MorganFingerprintTransformer(useFeatures=True)),
  ('standardscaler', StandardScaler()),
  ('xgbclassifier',
   XGBClassifier(base_score=None, booster=None, callbacks=None,
                 colsample_bylevel=None, colsample_bynode=None,
                 colsample_bytree=None, device=None, early_stopping_rounds=None,
                 enable_categorical=False, eval_metric=None, feature_types=None,
                 gamma=None, grow_policy=None, importance_type=None,
                 interaction_constraints=None, learning_rate=0.1, max_bin=None,
                 max_cat_threshold=None, max_cat_to_onehot=None,
                 max_delta_step=None, max_depth=3, max_leaves=None,
                 min_child_weight=None, missing=nan, monotone_constraints=None,
                 multi_strategy=None, n_estimators=100, n_jobs=None,
              

::: {.callout-note}
To see the default values or types of each XGBoost parameter, this XGBoost documentation [link](https://xgboost.readthedocs.io/en/latest/parameter.html#xgboost-parameters) is useful (which can be cross-referenced with XGBoost's [Python API reference](https://xgboost.readthedocs.io/en/latest/python/python_api.html#python-api-reference)).
:::

In [37]:
# To obtain only the parameter names for the ease of reading
mlpipe_xgb.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'smilestomoltransformer', 'standardizer', 'morganfingerprinttransformer', 'standardscaler', 'xgbclassifier', 'smilestomoltransformer__parallel', 'standardizer__neutralize', 'standardizer__parallel', 'morganfingerprinttransformer__nBits', 'morganfingerprinttransformer__parallel', 'morganfingerprinttransformer__radius', 'morganfingerprinttransformer__useBondTypes', 'morganfingerprinttransformer__useChirality', 'morganfingerprinttransformer__useCounts', 'morganfingerprinttransformer__useFeatures', 'standardscaler__copy', 'standardscaler__with_mean', 'standardscaler__with_std', 'xgbclassifier__objective', 'xgbclassifier__base_score', 'xgbclassifier__booster', 'xgbclassifier__callbacks', 'xgbclassifier__colsample_bylevel', 'xgbclassifier__colsample_bynode', 'xgbclassifier__colsample_bytree', 'xgbclassifier__device', 'xgbclassifier__early_stopping_rounds', 'xgbclassifier__enable_categorical', 'xgbclassifier__eval_metric', 'xgbclassifier__feature_types

Some of the main XGBoost parameters that can be tuned are *n_estimators*, *max_depth*, *learning_rate*, *subsample* and *reg_lambda*. Here, I'm going to try to look for the best combination of *learning_rate* and *subsample* for a XGBoost classifier model for now.

In [38]:
# Specify parameters and distributions to be sampled
params_dist = {
    # learning_rate usually between 0.01 - 0.1 as suggested by Raschka et al. 
    # default is between 0 and 1
    "xgbclassifier__learning_rate": [0.05, 0.1, 0.3], 
    # subsample default is between 0 and 1
    "xgbclassifier__subsample": [0.5, 0.7, 1.0]
}

<br>

###### **Randomised search CV**

The following chunk of code is an example of running randomised search CV. I've deliberately folded the code to minimise the reading length of the post and also because the result from it is very similar to the grid search CV used below (randomised search CV run time was 13 min 33.2 seconds due to having two pipelines containing two different machine learning models for the same set of data). It's being kept as a code reference for anyone who'd like to try it and also as an alternative way to do hyperparameter tuning. 

In [39]:
#| code-fold: true
## Uncomment code below to run
# from sklearn.model_selection import RandomizedSearchCV
# from time import time

## Borrowing a utility function code from scikit_learn documentation to report best scores
## Source: https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py

# def report(results, n_top=3):
#     for i in range(1, n_top + 1):
#         candidates = np.flatnonzero(results["rank_test_score"] == i)
#         for candidate in candidates:
#             print("Model with rank: {0}".format(i))
#             print(
#                 "Mean validation score: {0:.3f} (std: {1:.3f})".format(
#                     results["mean_test_score"][candidate],
#                     results["std_test_score"][candidate],
#                 )
#             )
#             print("Parameters: {0}".format(results["params"][candidate]))
#             print("")

## The following code has also referenced and adapted from this notebook 
## https://github.com/EBjerrum/scikit-mol/blob/main/notebooks/06_hyperparameter_tuning.ipynb

# n_iter_search = 9

# random_search = RandomizedSearchCV(
#     mlpipe_xgb, 
#     param_distributions=params_dist,
#     n_iter=n_iter_search,
#     n_jobs=2
# )

# t_start = time()
# random_search.fit(X_train, y_train)
# t_finish = time()

# print(f'Runtime: {t_finish-t_start:0.2F} seconds for {n_iter_search} iterations')

## Run report function code
# report(random_search.cv_results_)

<br>

###### **Grid search CV**

In [40]:
grid_search = GridSearchCV(
    mlpipe_xgb,
    param_grid=params_dist,
    verbose=1,
    n_jobs=2
)

grid_search.fit(X_train, y_train)

print(f"The best cv score is: {grid_search.best_score_:0.2f}")
print(f"The best cv parameter settings are: {grid_search.best_params_}")

# This may take longer time to run depending on computer hardware specs (for me it's taken ~13min)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
The best cv score is: 0.99
The best cv parameter settings are: {'xgbclassifier__learning_rate': 0.3, 'xgbclassifier__subsample': 0.7}


For tuning parameters of Morgan fingerprints, this Scikit_mol [example notebook](https://github.com/EBjerrum/scikit-mol/blob/main/notebooks/06_hyperparameter_tuning.ipynb) explains how to do it with code, so I won't repeat them here, but have only shown how to tune some of the main XGBoost parameters.

<br>

###### **Pickle model**

The next step is to pickle the model or pipeline if wanting to save it for future use and to avoid re-training model from the ground up again.

Some security tips regarding pickle module:

- [Python documentation](https://docs.python.org/3/library/pickle.html)
- [Reference blog post 1](https://www.synopsys.com/blogs/software-security/python-pickling.html) and [blog post 2](https://snyk.io/blog/guide-to-python-pickle/)

One thing to remember is to avoid unpickling unknown files over insecure network, and add security key if needed.

In [41]:
# Pickle to save (serialise) the model in working directory (specify path if needed)
# "wb" - write binary
pickle.dump(mlpipe_xgb, open("xgb_pipeline.pkl", "wb"))
# Unpickle (de-serialise) the model
# "rb" - read binary
mlpipe_xgb_2 = pickle.load(open("xgb_pipeline.pkl", "rb"))

# Use the unpickled model object to make prediction
pred2 = mlpipe_xgb_2.predict(X_test)

## Check unpickled model and original model are the same via Python's assertion method
#assert np.sum(np.abs(pred2 - pred)) == 0
## or alternatively use numpy's allclose()
print(np.allclose(pred, pred2))

True


<br>

##### **Acknowledgement**

Again, this grows into another really long post... Although this post has taken quite a long time to build up to completion, I still want to thank all the contributors or developers for all the packages used in this post.